In [97]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np 

In [216]:


dtype = np.float32

class Wavefunction:
    def __init__(self, alpha, beta):
        self.alpha = alpha 
        self.beta = beta
    def __call__(self, r):
        r1 = r[0]**2 + r[1]**2
        r2 = r[2]**2 + r[3]**2
        r12 = tf.sqrt((r[0]-r[2])**2 + (r[1]-r[3])**2)
        deno = r12/(1+self.beta*r12)
        return tf.math.log(tf.exp(-0.5*self.alpha*(r1+r2)+deno)**2)

def metropolis_step(state_parts, seed):
    uniform = np.random.uniform#tfp.distributions.Uniform()
    new_state_parts = []
    # print(len(state_parts))
    ran = np.random.randint(0, len(state_parts))
    print(state_parts)
    # for i, elem in enumerate(state_parts):
    #     print(i, ran)
    #     if i == ran:
    #         print("test1")
    #         new_state_parts.append(elem + np.random.uniform())
    #     else:
    #         print("test2")
    #         new_state_parts.append(elem)
    for elem in state_parts[0]:
        new_state_parts.append(elem)
        print("Elem", elem)
    print("DONE", new_state_parts)
    return new_state_parts




In [217]:
samples = tfp.mcmc.sample_chain(
    num_results =100, 
    current_state = np.array([0.1, 0, -0.1, 0]), 
    kernel = tfp.mcmc.RandomWalkMetropolis(
        target_log_prob_fn = Wavefunction(1.0,0.0), 
        new_state_fn= metropolis_step), 
    trace_fn=None)

[<tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 0.1,  0. , -0.1,  0. ])>]
Elem tf.Tensor(0.1, shape=(), dtype=float64)
Elem tf.Tensor(0.0, shape=(), dtype=float64)
Elem tf.Tensor(-0.1, shape=(), dtype=float64)
Elem tf.Tensor(0.0, shape=(), dtype=float64)
DONE [<tf.Tensor: shape=(), dtype=float64, numpy=0.1>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=-0.1>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>]


TypeError: __call__() takes 2 positional arguments but 5 were given

In [182]:
print(tf.math.reduce_mean( samples, axis=0))
print(samples)


tf.Tensor([1.0938285 0.9938285 0.8938285 0.9938285], shape=(4,), dtype=float64)
tf.Tensor(
[[ 0.1         0.         -0.1         0.        ]
 [ 0.33235548  0.23235548  0.13235548  0.23235548]
 [ 0.33235548  0.23235548  0.13235548  0.23235548]
 [ 0.33235548  0.23235548  0.13235548  0.23235548]
 [ 0.33235548  0.23235548  0.13235548  0.23235548]
 [ 0.33235548  0.23235548  0.13235548  0.23235548]
 [ 0.33235548  0.23235548  0.13235548  0.23235548]
 [ 0.33235548  0.23235548  0.13235548  0.23235548]
 [ 0.33235548  0.23235548  0.13235548  0.23235548]
 [ 0.46834064  0.36834064  0.26834064  0.36834064]
 [ 0.46834064  0.36834064  0.26834064  0.36834064]
 [ 0.46834064  0.36834064  0.26834064  0.36834064]
 [ 0.46834064  0.36834064  0.26834064  0.36834064]
 [ 0.46834064  0.36834064  0.26834064  0.36834064]
 [ 0.67508172  0.57508172  0.47508172  0.57508172]
 [ 0.67508172  0.57508172  0.47508172  0.57508172]
 [ 0.67508172  0.57508172  0.47508172  0.57508172]
 [ 0.67508172  0.57508172  0.47508172  0.5

In [3]:
mats = tf.random.uniform(shape=[1000, 10, 10])
vecs = tf.random.uniform(shape=[1000, 10, 1])

def vectorized_solve():
    return tf.linalg.solve(mats, vecs)

%timeit vectorized_solve()

487 µs ± 49.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
a = tf.Variable(np.pi)
b = tf.Variable(np.e)

with tf.GradientTape() as tape:
    # tape.watch([a,b])
    c = 0.5*(a**2 + b**2)
grads = tape.gradient(c, [a,b])
print([var.name for var in tape.watched_variables()])
print(grads[0])


['Variable:0', 'Variable:0']
tf.Tensor(3.1415927, shape=(), dtype=float32)


In [20]:

@tf.function
def test_func2(a,b):
    for i in tf.range(10000):
        tf.tensordot(a, b, axes = 0)

a = tf.Variable([100, 100])
b = tf.Variable([100,100])
%timeit test_func2(a,b)



121 ms ± 5.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
